Update investment-options.csv manually first, then run this notebook.

See https://www.myfrs.com/InvestmentFundTabs.htm

Values as of Jan 29, 2023

In [6]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [7]:
import matplotlib.pyplot as plt
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [8]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [9]:
filename = 'investment-options.csv'

In [10]:
# Read in investment options input csv.
df = pd.read_csv(filename, skip_blank_lines=True, comment='#')
df

,Investment Option,Asset Class,1 Yr,3 Yr,5 Yr,SD 1 Yr,SD 3 Yr,SD 5 Yr
0,FRS Inflation Sensitive Fund (300),US Bonds:Treasury Inflation-Protected Securiti...,-7.70,2.70,2.94,10.87,10.87,10.87
1,FRS U.S. Bond Enhanced Index Fund (80),US Bonds,-13.11,-2.71,3.25,5.94,5.94,5.94
2,FRS Core Plus Bond Fund (310),US Bonds,-13.16,-1.97,0.80,6.92,6.92,6.92
3,FRS U.S. Stock Market Index Fund (120),US Stocks,-19.19,7.11,8.84,21.79,21.79,21.79
4,FRS U.S. Stock Fund (340),US Stocks,-22.39,4.31,6.12,24.02,24.02,24.02
5,FRS Foreign Stock Index Fund (200),Global Stocks,-16.58,0.34,1.06,20.41,20.41,20.41
6,FRS Global Stock Fund (210),Global Stocks,-25.64,5.51,7.67,22.43,22.43,22.43
7,FRS Foreign Stock Fund (220),Global Stocks,-22.72,-0.15,1.54,21.45,21.45,21.45
8,FRS Stable Value Fund (350),Cash Equivalents,1.84,1.93,2.06,0.08,0.08,0.08
